# LangSmith Integration (Bonus) <a id='langsmith'></a>

## 1. Testing with Sample Queries <a id='testing'></a>

Let's test our complete chain with the suggested test queries.

In [ ]:
# Define test queries as specified in the requirements
test_queries = [
    {
        "name": "Neutral-Informative",
        "query": "Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago",
        "expected_category": "product_inquiry"
    },
    {
        "name": "Urgent-Negative",
        "query": "This is an emergency! My order #TEC-2024001 never arrived and I need that laptop for work tomorrow!",
        "expected_category": "billing"
    },
    {
        "name": "Satisfied-Positive",
        "query": "Thank you so much for the excellent service with my previous purchase, I want to buy gaming headphones",
        "expected_category": "product_inquiry"
    },
    {
        "name": "Frustrated-Technical",
        "query": "I can't configure the router I bought last week, I've tried everything and it doesn't work",
        "expected_category": "technical_support"
    },
    {
        "name": "Formal-Billing",
        "query": "Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023089",
        "expected_category": "billing"
    },
    {
        "name": "Warranty-Query",
        "query": "I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?",
        "expected_category": "returns"
    }
]

print(f"🧪 Ready to test {len(test_queries)} sample queries")

In [ ]:
# Environment setup
import os
import sys
from typing import Literal, Optional, List, Dict, Any
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get the notebook directory and project root
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
project_root = os.path.dirname(notebook_dir)

# Add project root to Python path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"📁 Project root: {project_root}")

# Import CustomerServiceChain
from src.chains import CustomerServiceChain

print("✅ CustomerServiceChain imported successfully!")

## 2. LangSmith Dataset

Create an evaluation dataset in LangSmith for tracking performance.

In [ ]:

# Function to create evaluation dataset in LangSmith
# Note: This requires LangSmith API access

def create_langsmith_dataset():
    """
    Creates an evaluation dataset in LangSmith with our test queries.
    Uncomment and run if you have LangSmith configured.
    """
    try:
        from langsmith import Client
        
        # Initialize LangSmith client
        client = Client()
        
        # Create dataset
        dataset_name = "customer-service-queries-v1"
        dataset = client.create_dataset(
            dataset_name=dataset_name,
            description="Test queries for customer service agent evaluation"
        )
        
        # Add examples to the dataset
        for test in test_queries:
            client.create_example(
                inputs={"query": test["query"]},
                outputs={"expected_category": test["expected_category"]},
                dataset_id=dataset.id
            )
        
        print(f"✅ Created dataset '{dataset_name}' with {len(test_queries)} examples")
        print(f"📊 View in LangSmith: https://smith.langchain.com/datasets/{dataset.id}")
        
        return dataset
        
    except Exception as e:
        print(f"⚠️  LangSmith integration not available: {str(e)}")
        print("Make sure you have:")
        print("1. LangSmith API key configured")
        print("2. langsmith package installed")
        return None

# Uncomment to create dataset
# dataset = create_langsmith_dataset()

In [ ]:
# Display information about LangSmith tracing
print("📊 LANGSMITH TRACING INFORMATION")
print("=" * 50)
print(f"Tracing Enabled: {os.environ.get('LANGSMITH_TRACING', 'false')}")
print(f"Project Name: {os.environ.get('LANGSMITH_PROJECT', 'Not set')}")
print(f"\n🔗 View traces at: https://smith.langchain.com")
print("\nTo view traces:")
print("1. Ensure you have a LangSmith account")
print("2. Set your LANGSMITH_API_KEY environment variable")
print("3. Run queries and check the LangSmith dashboard")

## 3. Performance Analysis <a id='analysis'></a>

Analyze the performance of our chain based on test results.

In [ ]:
def process_customer_query(query: str) -> Dict[str, Any]:
    agent = CustomerServiceChain()
    return agent.process_query(query)

In [ ]:
# Run all test queries and collect results
test_results = []

print("=== RUNNING TEST QUERIES ===")
print("\n" + "="*80 + "\n")

for i, test in enumerate(test_queries, 1):
    print(f"📋 TEST {i}/{len(test_queries)}: {test['name']}")
    print(f"📝 Query: {test['query']}")
    print("-" * 80)
    
    # Process the query
    result = process_customer_query(test['query'])
    
    if result:
        test_results.append(result)        
        print(f"\n✅ Agent Response: {result['response']}")
        
    else:
        print("❌ Failed to process query")
    
    print("\n" + "="*80 + "\n")

print(f"✅ Completed testing {len(test_results)}/{len(test_queries)} queries successfully!")

In [ ]:
# Performance analysis function
def analyze_chain_performance(results: List[Dict[str, Any]]):
    """
    Analyzes the performance of the chain based on test results.
    """
    if not results:
        print("No results to analyze")
        return
    
    print("\n📈 PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    # Initialize counters
    categories = {}
    sentiments = {}
    urgencies = {}
    statuses = {}
    
    # Count occurrences
    for result in results:
        summary = result['summary']
        
        # Count categories
        cat = summary.query_category
        categories[cat] = categories.get(cat, 0) + 1
        
        # Count sentiments
        sent = summary.customer_sentiment
        sentiments[sent] = sentiments.get(sent, 0) + 1
        
        # Count urgency levels
        urg = summary.urgency_level
        urgencies[urg] = urgencies.get(urg, 0) + 1
        
        # Count resolution statuses
        status = summary.resolution_status
        statuses[status] = statuses.get(status, 0) + 1
    
    # Display results
    print("\n📊 Category Distribution:")
    for cat, count in sorted(categories.items()):
        percentage = (count / len(results)) * 100
        print(f"  - {cat}: {count} ({percentage:.1f}%)")
    
    print("\n😊 Sentiment Distribution:")
    for sent, count in sorted(sentiments.items()):
        percentage = (count / len(results)) * 100
        print(f"  - {sent}: {count} ({percentage:.1f}%)")
    
    print("\n⚡ Urgency Distribution:")
    for urg, count in sorted(urgencies.items()):
        percentage = (count / len(results)) * 100
        print(f"  - {urg}: {count} ({percentage:.1f}%)")
    
    print("\n📋 Resolution Status Distribution:")
    for status, count in sorted(statuses.items()):
        percentage = (count / len(results)) * 100
        print(f"  - {status}: {count} ({percentage:.1f}%)")
    
    # Calculate follow-up rate
    follow_ups = sum(1 for r in results if r['summary'].follow_up_required)
    follow_up_rate = (follow_ups / len(results)) * 100
    print(f"\n🔄 Follow-up Required: {follow_ups}/{len(results)} ({follow_up_rate:.1f}%)")
    
    print("\n✅ Analysis Complete!")

# Run the analysis
analyze_chain_performance(test_results)